In [2]:
import pandas as pd
import numpy as np
agency = pd.read_csv('../datasets/asana/gtfs_data/agency.txt')
routes = pd.read_csv('../datasets/asana/gtfs_data/routes_utf8.txt', sep="\t")
stop_times = pd.read_csv('../datasets/asana/gtfs_data/stop_times.txt', sep="\t")
stops = pd.read_csv('../datasets/asana/gtfs_data/stops.txt', sep="\t")
trips = pd.read_csv('../datasets/asana/gtfs_data/trips.txt', sep="\t")



In [71]:
modeling_structure_df = pd.DataFrame()

route_travel_time = pd.DataFrame()
arrival_time = pd.DataFrame()
delay_prediction = pd.DataFrame()
# most descriptive columns from the gtfs format for modeling that i currently identified are:
#  from stop_times : stop_id, arrival_time, departure_time
#  from stops : stop_id, stop_lat, stop_lon
#  from trips : shape_id, route_id (To relate the info, basically a join table)

# The features that i imagine i could extract to get more info are:

# average_stop_times (Would describe an average of time from stop to stop per route)
# a more defined and specific attributes below:

#avg_seconds_between_stops
#std_seconds_between_stops

# hour_of_day (Pretty descriptive name)
# stop_quantity (A sum of the bus stops)
# stop_density_500m (This measures how conglomerated a zone is in the 500m radious)


# gps_data = pd.read_csv('../datasets/asana/gps_data.csv') # thing is so big i need to put attributes here because checking takes long - id,deviceid,devicetime,latitude,longitude,speed,route_direction,route_id,route_guid,device_guid

# Fordward here im going to explain the structure of how this dataset is going to be analized

# I'm going to add steps to it: 
# 1st Step is going to ananlyze just structural, coordinates, stop density, etc. So i can see for example if using a delay in time target (Which will probably be the most used target in this notebook), i can see how much the data is giving in comparison 
# ------- (Here on step 1, will be using normal train_test_split and will be trained with diferent models and compared with CV)
# to 2nd Step which will only use context data such as speeds, times, etc. || i will search for most significant/descriptive attributes for each, and look into compare important stadistic values such as R^2 and others
# ------- (Here on step 2, will be using time_series_split with lags, so the model also finds relationship in between the difference in time)
# Finally on step 3, depending on which attributes tend to be the most meaningful and tune the best models to find a in between in overfitting and underfitting 


# Now onto other other targets its going to be the effective_speeds
# Using the GTFS-RT, finding the actual time taken to complete a route, we can find the delays in the actual schedulet time, which will be useful to create a "isDelayed" target using a specific treshold
# Clustering could be applied using the times, the route and locations and stop densities to find congestion zones



In [72]:
# First row im going to compute is the trip length in KM with sets of longitudes using the Haversine formula |||| reference : https://www.youtube.com/watch?v=cD6pUr_5RtE

In [73]:
import math

def distance_between(lat1, lon1, lat2, lon2):
    earthRadius = 6371
    dlat = degreesToRadians(lat2-lat1)
    dlon = degreesToRadians(lon2-lon1) 
    rad = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(degreesToRadians(lat1)) * math.cos(degreesToRadians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    rad = 2 * math.atan2(math.sqrt(rad), math.sqrt(1-rad))
    distance = earthRadius * rad
    return distance
    
def degreesToRadians(deg):
    return deg * (math.pi / 180) 

In [74]:
#stops.head()


In [57]:
trips = trips.sort_values(by=["vehicle_id", "start_time"])

In [58]:
trips

,route_id,service_id,trip_id,direction_id,start_time,end_time,vehicle_id
15961,c453217f-9f9d-49ce-8b32-14a6b7013691,service_2024-08-12,8409,1,06:00:00,07:27:29,010670bd-20f2-485a-b2d5-f4a5fae53208
15993,c453217f-9f9d-49ce-8b32-14a6b7013691,service_2024-08-15,10396,2,06:00:00,07:57:14,010670bd-20f2-485a-b2d5-f4a5fae53208
16025,c453217f-9f9d-49ce-8b32-14a6b7013691,service_2024-08-19,12744,1,06:00:00,07:53:20,010670bd-20f2-485a-b2d5-f4a5fae53208
16104,c453217f-9f9d-49ce-8b32-14a6b7013691,service_2024-08-26,17186,1,06:00:00,07:01:48,010670bd-20f2-485a-b2d5-f4a5fae53208
16119,c453217f-9f9d-49ce-8b32-14a6b7013691,service_2024-08-27,17850,1,06:00:00,07:16:06,010670bd-20f2-485a-b2d5-f4a5fae53208
...,...,...,...,...,...,...,...
7564,d626b854-27aa-41d4-8625-ebafa73d8f21,service_2024-08-20,10207,1,19:46:50,21:09:44,fc828ac8-2318-49d9-bb73-a9b1de024469
7539,d626b854-27aa-41d4-8625-ebafa73d8f21,service_2024-09-13,20791,1,20:02:59,21:24:16,fc828ac8-2318-49d9-bb73-a9b1de024469
7584,d626b854-27aa-41d4-8625-ebafa73d8f21,service_2024-08-09,5361,1,20:03:32,21:17:07,fc828ac8-2318-49d9-bb73-a9b1de024469
7578,d626b854-27aa-41d4-8625-ebafa73d8f21,service_2024-08-12,6611,1,20:08:05,21:11:54,fc828ac8-2318-49d9-bb73-a9b1de024469


In [21]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,10406,07:20:44,07:20:44,5d11962d-6412-4c7c-984d-f4ee5eb63859,41
1,27104,20:14:37,20:14:57,5d11962d-6412-4c7c-984d-f4ee5eb63859,40
2,25687,17:45:24,17:45:43,5d11962d-6412-4c7c-984d-f4ee5eb63859,41
3,25541,10:42:59,10:42:59,5d11962d-6412-4c7c-984d-f4ee5eb63859,42
4,22444,13:40:10,13:41:00,5d11962d-6412-4c7c-984d-f4ee5eb63859,35


In [77]:
#routes.head()

In [78]:
sorted_stop_times = stop_times.sort_values(by=['trip_id', 'stop_sequence'], ascending=[True, True])


In [79]:
sorted_route_stops_coords = sorted_stop_times.merge(stops[["stop_id", "stop_lat", "stop_lon"]], on="stop_id", how="left")


In [80]:
sorted_route_stops_coords["prev_lat"] = sorted_route_stops_coords["stop_lat"].shift(1)
sorted_route_stops_coords["prev_lon"] = sorted_route_stops_coords["stop_lon"].shift(1)


In [81]:
sorted_route_stops_coords["segment_distance"] = sorted_route_stops_coords.apply(
    lambda row: distance_between(
        row["prev_lat"],
        row["prev_lon"],
        row["stop_lat"],
        row["stop_lon"]
    ) if pd.notnull(row["prev_lat"]) else 0,
    axis=1
)


In [82]:
route_distance_df = (
    sorted_route_stops_coords.groupby("trip_id")["segment_distance"]
      .sum()
      .reset_index(name="total_distance_km")
)


In [83]:
modeling_structure_df = trips

In [84]:

modeling_structure_df = modeling_structure_df.drop(columns=['service_id','vehicle_id', 'direction_id'])

In [85]:
modeling_structure_df = modeling_structure_df.merge(route_distance_df, on="trip_id", how="left")


In [86]:
stop_count = (
    stop_times.groupby("trip_id")['stop_sequence'].count()
    .reset_index(name="total_stops")
)

In [87]:
modeling_structure_df = modeling_structure_df.merge(stop_count, on="trip_id", how="left")


In [88]:
def calcEffectiveSpeed(start_time_s, end_time_s, total_distance_km):
    if end_time_s < start_time_s:  #this means it cycled to next day (which i find improbable, but still)
        end_time_s+=86400
    duration = end_time_s - start_time_s

    if duration <= 0:
        return np.nan #invalid data
    
    return total_distance_km * 3600/duration
    

In [89]:
modeling_structure_df["start_sec"] = pd.to_timedelta(modeling_structure_df["start_time"]).dt.total_seconds()
modeling_structure_df["end_sec"]   = pd.to_timedelta(modeling_structure_df["end_time"]).dt.total_seconds()
modeling_structure_df = modeling_structure_df.drop(columns=['start_time', 'end_time']) 

modeling_structure_df["effective_scheduled_speed_kmh"] = modeling_structure_df.apply(
    lambda row: calcEffectiveSpeed(
        row["start_sec"],
        row["end_sec"],
        row["total_distance_km"],
    ),
    axis=1
)


In [90]:
modeling_structure_df

,route_id,trip_id,total_distance_km,total_stops,start_sec,end_sec,effective_scheduled_speed_kmh
0,d626b854-27aa-41d4-8625-ebafa73d8f21,23809,37.516956,36,38521.0,44990.0,20.878195
1,d626b854-27aa-41d4-8625-ebafa73d8f21,23705,37.516956,36,28733.0,33312.0,29.495751
2,d626b854-27aa-41d4-8625-ebafa73d8f21,23547,21.245568,39,70904.0,75385.0,17.068522
3,d626b854-27aa-41d4-8625-ebafa73d8f21,23497,21.077345,36,62522.0,70901.0,9.055787
4,d626b854-27aa-41d4-8625-ebafa73d8f21,23455,21.353914,40,57835.0,62517.0,16.419071
...,...,...,...,...,...,...,...
19764,c453217f-9f9d-49ce-8b32-14a6b7013691,26011,38.988103,44,53273.0,58278.0,28.043391
19765,c453217f-9f9d-49ce-8b32-14a6b7013691,25978,20.239318,38,48387.0,53259.0,14.955161
19766,c453217f-9f9d-49ce-8b32-14a6b7013691,25951,25.623639,44,43346.0,48379.0,18.328055
19767,c453217f-9f9d-49ce-8b32-14a6b7013691,25902,36.667677,39,36459.0,43292.0,19.318548


In [23]:
routes_per_stop_df = (stop_times.groupby("stop_id")["trip_id"].count().reset_index(name="routes_per_stop"))
routes_per_stop_df["routes_per_stop"] = routes_per_stop_df["routes_per_stop"] - 1 #exclude current route 


merged = stop_times.merge(routes_per_stop_df, on="stop_id", how="left")

In [24]:
overlap_score = (
    merged.groupby('trip_id')["routes_per_stop"]
    .mean()
    .reset_index(name="overlap_score")
)

In [25]:
modeling_structure_df = modeling_structure_df.merge(overlap_score, on="trip_id", how="left")

In [26]:
modeling_structure_df["stop_density"] = (modeling_structure_df["total_stops"]-1) / modeling_structure_df["total_distance_km"] #substracting 1 to stops to measure the actual segments between stops

In [27]:
modeling_structure_df = modeling_structure_df.drop(columns=["total_stops", "start_sec", "end_sec"])

In [69]:
modeling_structure_df 

NameError: name 'modeling_structure_df' is not defined

In [29]:
modeling_structure_df[["total_distance_km", "stop_density", "overlap_score", "effective_scheduled_speed_kmh"]].corr() # stop density and total distance km are very correlated, but will keep it because 
                                                                                                                      #i will probably just use gradient boosting and it already handles it


,total_distance_km,stop_density,overlap_score,effective_scheduled_speed_kmh
total_distance_km,1.000000,-0.934410,-0.062129,0.700862
stop_density,-0.934410,1.000000,0.141396,-0.632685
overlap_score,-0.062129,0.141396,1.000000,-0.216111
effective_scheduled_speed_kmh,0.700862,-0.632685,-0.216111,1.000000


In [30]:
modeling_structure_df = modeling_structure_df.drop(columns=["route_id", "trip_id"])

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, KFold

X = modeling_structure_df.drop(columns=["effective_scheduled_speed_kmh"])
y = modeling_structure_df["effective_scheduled_speed_kmh"]


In [32]:
linearRegression_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

In [33]:
xgb_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("model", XGBRegressor(
    n_estimators=200,
    learning_rate=0.3,
    max_depth=6,
    ))
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2
)
# First training with just structural attributes

xgb_scores = cross_val_score(xgb_pipeline, X_train, y_train, cv=5)
xgb_scores_mae = cross_val_score(xgb_pipeline, X_train, y_train, cv=5,scoring="neg_mean_absolute_error")
xgb_scores_rmae = cross_val_score(xgb_pipeline, X_train, y_train, cv=5,scoring='neg_mean_squared_error')
LR_scores = cross_val_score(linearRegression_pipeline, X_train, y_train, cv=5)
LR_scores_mae = cross_val_score(linearRegression_pipeline, X_train, y_train, cv=5,scoring="neg_mean_absolute_error")
LR_scores_rmae = cross_val_score(linearRegression_pipeline, X_train, y_train, cv=5,scoring='neg_mean_squared_error')

In [34]:
print(f"xgb R^2: {xgb_scores.mean()}, MAE: {xgb_scores_mae.mean()} RMAE: {xgb_scores_rmae.mean()}")
print(f"LR R^2: {LR_scores.mean()}, MAE: {LR_scores_mae.mean()}RMAE: {LR_scores_rmae.mean()}")


xgb R^2: 0.5791297142081241, MAE: -2.774794700912145 RMAE: -14.852166950045037
LR R^2: 0.535276694226706, MAE: -3.0338633987924446RMAE: -16.40364862954919


In [35]:
# now feature engineering with time data GTFS-RT data: 
# Columns : id,deviceid,devicetime,latitude,longitude,speed,route_direction,route_id,route_guid,device_guid

In [36]:
# here will use duck db because making operations on a 115 Million ish table sucks with low ram

In [2]:
import duckdb

con = duckdb.connect()





In [91]:

con.register("modeling_structure_df", modeling_structure_df)

con.execute("""
COPY modeling_structure_df TO 'modeling_structure_df.parquet' (FORMAT PARQUET);
""")

In [ ]:
con.execute("""
    CREATE TABLE gps_sorted AS
    SELECT *
    FROM 'gps_clean.parquet'
    ORDER BY deviceid, devicetime
""")


In [3]:
con.execute("""
    CREATE TABLE segments AS
    SELECT * FROM read_csv_auto('../datasets/asana/segment_level_data.csv');
""")

In [4]:
con.execute("DESCRIBE gps_sorted").fetchdf()


,column_name,column_type,null,key,default,extra
0,id,BIGINT,YES,None,None,None
1,deviceid,BIGINT,YES,None,None,None
2,devicetime,TIMESTAMP,YES,None,None,None
3,latitude,DOUBLE,YES,None,None,None
4,longitude,DOUBLE,YES,None,None,None
5,speed,DOUBLE,YES,None,None,None
6,route_direction,VARCHAR,YES,None,None,None
7,route_id,BIGINT,YES,None,None,None
8,route_guid,VARCHAR,YES,None,None,None
9,device_guid,VARCHAR,YES,None,None,None


In [5]:
con.execute("""
    COPY (
        SELECT *
        FROM read_csv_auto('../datasets/asana/segment_level_data.csv')
    )
    TO 'segments.parquet'
    (FORMAT PARQUET);
""")

In [6]:
con.execute("DESCRIBE segments").fetchdf()


,column_name,column_type,null,key,default,extra
0,date,DATE,YES,None,None,None
1,deviceid,BIGINT,YES,None,None,None
2,direction,BIGINT,YES,None,None,None
3,segment,BIGINT,YES,None,None,None
4,start_point,BIGINT,YES,None,None,None
5,end_point,BIGINT,YES,None,None,None
6,start_time,TIMESTAMP,YES,None,None,None
7,run_time_in_seconds,BIGINT,YES,None,None,None
8,dwell_time_in_seconds,BIGINT,YES,None,None,None
9,arrival_time,TIMESTAMP,YES,None,None,None


In [7]:
con.execute("DESCRIBE segments").fetchdf()


,column_name,column_type,null,key,default,extra
0,date,DATE,YES,None,None,None
1,deviceid,BIGINT,YES,None,None,None
2,direction,BIGINT,YES,None,None,None
3,segment,BIGINT,YES,None,None,None
4,start_point,BIGINT,YES,None,None,None
5,end_point,BIGINT,YES,None,None,None
6,start_time,TIMESTAMP,YES,None,None,None
7,run_time_in_seconds,BIGINT,YES,None,None,None
8,dwell_time_in_seconds,BIGINT,YES,None,None,None
9,arrival_time,TIMESTAMP,YES,None,None,None


In [8]:
test = con.execute(""" SELECT
                trip_id,
                EXTRACT(EPOCH FROM MAX(start_time) - MIN(arrival_time)) AS scheduled_duration,
                FROM segments
                WHERE trip_id = 2869
                GROUP BY trip_id""").fetchdf()

In [9]:
test

,trip_id,scheduled_duration
0,2869,3879.0


In [59]:
segments_from_trip = con.execute("""
    SELECT
        s.trip_id,
        s.start_time,
        s.arrival_time,
        AVG(g.speed) AS avg_speed,
        VAR_SAMP(g.speed) AS variance,
        EXTRACT(EPOCH FROM MAX(g.devicetime) - MIN(g.devicetime)) AS real_duration
    
        
    FROM gps_sorted g
    JOIN segments s
      ON g.deviceid = s.deviceid
     AND g.devicetime BETWEEN s.start_time AND s.arrival_time

    GROUP BY
        s.trip_id,
        s.start_time,
        s.arrival_time
""").fetchdf()

In [60]:
segments_from_trip

,trip_id,start_time,arrival_time,avg_speed,variance,real_duration
0,6507,2024-08-12 17:46:01,2024-08-12 17:46:36,18.788351,192.806425,35.0
1,6611,2024-08-12 20:58:54,2024-08-12 21:00:23,19.443817,121.799770,89.0
2,6611,2024-08-12 20:27:11,2024-08-12 20:27:58,16.566512,18.507282,47.0
3,6611,2024-08-12 21:07:22,2024-08-12 21:08:09,8.517198,25.644583,47.0
4,6981,2024-08-13 17:32:56,2024-08-13 17:38:03,7.201395,32.023550,307.0
...,...,...,...,...,...,...
785969,28418,2024-09-18 19:53:15,2024-09-18 19:54:18,16.038703,172.708357,63.0
785970,28530,2024-09-19 09:05:45,2024-09-19 09:06:26,27.041528,218.583459,41.0
785971,28530,2024-09-19 09:22:23,2024-09-19 09:22:59,16.026478,117.984805,36.0
785972,28530,2024-09-19 09:11:05,2024-09-19 09:13:34,14.045669,159.341579,149.0


In [17]:
segments_from_trip[segments_from_trip["trip_id"] == 8000].sort_values(by=["start_time"])

,trip_id,start_time,arrival_time,avg_speed,variance,real_duration
549891,8000,2024-08-11 09:02:22,2024-08-11 09:11:06,20.344864,67.470811,524.0
608900,8000,2024-08-11 09:19:50,2024-08-11 09:22:08,10.858886,116.110706,138.0
358971,8000,2024-08-11 09:22:50,2024-08-11 09:23:53,23.273490,97.617764,63.0
545944,8000,2024-08-11 09:24:08,2024-08-11 09:24:49,23.388337,43.202178,41.0
358972,8000,2024-08-11 09:24:58,2024-08-11 09:26:04,24.024719,31.517147,66.0
314428,8000,2024-08-11 09:26:16,2024-08-11 09:29:08,14.629116,70.447776,172.0
723826,8000,2024-08-11 09:29:13,2024-08-11 09:30:18,17.694080,32.785172,65.0
545943,8000,2024-08-11 09:30:44,2024-08-11 09:31:23,28.655235,130.573313,39.0
314427,8000,2024-08-11 09:31:46,2024-08-11 09:32:28,23.017905,117.588610,42.0
449868,8000,2024-08-11 09:32:50,2024-08-11 09:33:48,25.722677,115.847607,58.0


In [61]:
con.execute("""
    CREATE OR REPLACE TABLE trip_segments_full AS
    SELECT * FROM segments_from_trip
""")

In [ ]:
con.execute("""
    COPY (
        SELECT *
        FROM read_csv_auto('../datasets/asana/gtfs_data/trips.txt')
    )
    TO 'trips.parquet'
    (FORMAT PARQUET);
""")

In [ ]:
con.execute("""
    CREATE TABLE trips AS
    SELECT * FROM read_parquet('trips.parquet')
""")

In [ ]:
con.execute("""
COPY (SELECT *, EXTRACT(EPOCH FROM end_time) - EXTRACT(EPOCH FROM start_time) AS scheduled_duration FROM trips) TO 'trip_tweak.parquet' (FORMAT PARQUET);
""")


In [ ]:
df = con.execute("SELECT trip_id, scheduled_duration FROM trip_tweak.parquet WHERE trip_id = 2869" ).fetchdf()
df

In [20]:
df = con.execute("SELECT * FROM trip_segments_full").fetchdf()


In [28]:
df

,trip_id,trip_real_duration,trip_avg_speed
0,7065,3175.0,16.341221
1,5923,3551.0,20.728096
2,11159,4463.0,15.701129
3,11275,2753.0,23.086438
4,15627,2990.0,27.420993
...,...,...,...
19764,26322,3749.0,19.161470
19765,4164,3172.0,20.619375
19766,23616,3006.0,16.771271
19767,5258,3506.0,18.340325


In [62]:
con.execute("""CREATE OR REPLACE TABLE trip_segments_full AS 
SELECT
    trip_id,
    SUM(real_duration) AS trip_real_duration,
    SUM(avg_speed * real_duration) / SUM(real_duration) AS trip_avg_speed,
    AVG(variance) as avg_variance,
    MIN(start_time) as start_datetime,
    MAX(arrival_time) as end_datetime
FROM trip_segments_full
GROUP BY trip_id""");


In [63]:
df = con.execute("SELECT * FROM trip_segments_full").fetchdf()


In [64]:
df

,trip_id,trip_real_duration,trip_avg_speed,avg_variance,start_datetime,end_datetime
0,9633,4867.0,15.618442,76.544392,2024-08-13 18:54:32,2024-08-13 20:31:47
1,10336,3086.0,16.991997,65.321028,2024-08-14 20:40:09,2024-08-14 21:43:17
2,11490,6773.0,15.321919,85.261477,2024-08-16 15:17:37,2024-08-16 18:02:56
3,11782,3322.0,17.375570,74.624023,2024-08-17 08:09:10,2024-08-17 09:19:13
4,5212,4036.0,18.395965,98.269678,2024-08-06 15:13:51,2024-08-06 16:40:55
...,...,...,...,...,...,...
19764,26667,3922.0,15.927480,101.851187,2024-09-13 13:24:11,2024-09-13 14:47:32
19765,17594,3756.0,16.981150,85.469122,2024-08-26 15:28:07,2024-08-26 16:46:57
19766,24476,3897.0,14.817847,79.112279,2024-09-06 19:06:55,2024-09-06 20:25:24
19767,18387,4578.0,18.834880,111.540439,2024-09-08 18:44:55,2024-09-08 20:42:48


In [40]:
trip_tweak = con.execute("""SELECT * FROM trip_tweak.parquet""").fetchdf()

In [41]:
trip_tweak

,route_id,service_id,trip_id,direction_id,start_time,end_time,vehicle_id,scheduled_duration
0,d626b854-27aa-41d4-8625-ebafa73d8f21,service_2024-09-20,23809,1,10:42:01,12:29:50,07f4d1a4-b216-4906-a4b7-5a3d7af38266,6469.0
1,d626b854-27aa-41d4-8625-ebafa73d8f21,service_2024-09-20,23705,1,07:58:53,09:15:12,07f4d1a4-b216-4906-a4b7-5a3d7af38266,4579.0
2,d626b854-27aa-41d4-8625-ebafa73d8f21,service_2024-09-19,23547,2,19:41:44,20:56:25,07f4d1a4-b216-4906-a4b7-5a3d7af38266,4481.0
3,d626b854-27aa-41d4-8625-ebafa73d8f21,service_2024-09-19,23497,1,17:22:02,19:41:41,07f4d1a4-b216-4906-a4b7-5a3d7af38266,8379.0
4,d626b854-27aa-41d4-8625-ebafa73d8f21,service_2024-09-19,23455,2,16:03:55,17:21:57,07f4d1a4-b216-4906-a4b7-5a3d7af38266,4682.0
...,...,...,...,...,...,...,...,...
19764,c453217f-9f9d-49ce-8b32-14a6b7013691,service_2024-09-11,26011,2,14:47:53,16:11:18,2b0be505-6e54-4389-bfdf-ade82125ec13,5005.0
19765,c453217f-9f9d-49ce-8b32-14a6b7013691,service_2024-09-11,25978,1,13:26:27,14:47:39,2b0be505-6e54-4389-bfdf-ade82125ec13,4872.0
19766,c453217f-9f9d-49ce-8b32-14a6b7013691,service_2024-09-11,25951,2,12:02:26,13:26:19,2b0be505-6e54-4389-bfdf-ade82125ec13,5033.0
19767,c453217f-9f9d-49ce-8b32-14a6b7013691,service_2024-09-11,25902,1,10:07:39,12:01:32,2b0be505-6e54-4389-bfdf-ade82125ec13,6833.0


In [65]:
final_time_df = con.execute(""" 
    SELECT 
        s.trip_id,
        s.trip_real_duration,
        s.trip_avg_speed,
        (s.trip_real_duration - t.scheduled_duration) AS delay,
        s.avg_variance,
        t.direction_id,
        s.start_datetime,
        s.end_datetime

    FROM trip_segments_full s
    JOIN trip_tweak.parquet t
      ON s.trip_id = t.trip_id
""").fetchdf()

In [68]:
con.register("final_time_df", final_time_df)

con.execute("""
COPY final_time_df TO 'final_time_df.parquet' (FORMAT PARQUET);
""")

In [58]:
(trips_data["delay"] > 0).sum()

np.int64(50)

In [49]:
trips_data["delay"].median()
trips_data["delay"].quantile([0.1, 0.25, 0.75, 0.9])

0.10   -2935.2
0.25   -1733.0
0.75   -1177.0
0.90   -1034.0
Name: delay, dtype: float64

In [50]:
trips_data.sort_values("delay").head(10)
trips_data.sort_values("delay", ascending=False).head(10)

,trip_id,trip_real_duration,trip_avg_speed,delay,avg_variance
3411,17452,19502.0,14.386459,6273.0,93.341256
7358,19563,18396.0,17.511511,4323.0,97.766275
16174,2500,13052.0,9.910273,3741.0,93.421056
19083,15422,14969.0,8.635050,3715.0,88.932891
16867,19590,15044.0,12.530933,3157.0,81.229643
3233,10401,14363.0,11.004349,3095.0,97.294800
15398,7573,8176.0,26.101895,2933.0,156.130654
4972,2715,9395.0,25.425699,2903.0,108.442580
75,16059,8119.0,24.116174,2399.0,95.917179
18273,26446,11579.0,17.767479,2066.0,160.581249


In [54]:
print(trips_data["trip_real_duration"].mean())
print(trip_tweak["scheduled_duration"].mean())

4188.967524912742
5899.5189438009


In [55]:
trip_tweak["scheduled_duration"].describe()
trips_data["trip_real_duration"].describe()

count    19769.000000
mean      4188.967525
std       1080.995323
min       1017.000000
25%       3448.000000
50%       3918.000000
75%       4758.000000
max      19502.000000
Name: trip_real_duration, dtype: float64

In [95]:
final_time_df

,trip_id,trip_real_duration,trip_avg_speed,delay,avg_variance,direction_id,start_datetime,end_datetime
0,9633,4867.0,15.618442,-1038.0,76.544392,2,2024-08-13 18:54:32,2024-08-13 20:31:47
1,10336,3086.0,16.991997,-798.0,65.321028,1,2024-08-14 20:40:09,2024-08-14 21:43:17
2,11490,6773.0,15.321919,-1171.0,85.261477,1,2024-08-16 15:17:37,2024-08-16 18:02:56
3,11782,3322.0,17.375570,-1190.0,74.624023,1,2024-08-17 08:09:10,2024-08-17 09:19:13
4,5212,4036.0,18.395965,-1173.0,98.269678,2,2024-08-06 15:13:51,2024-08-06 16:40:55
...,...,...,...,...,...,...,...,...
19764,26667,3922.0,15.927480,-1241.0,101.851187,2,2024-09-13 13:24:11,2024-09-13 14:47:32
19765,17594,3756.0,16.981150,-1296.0,85.469122,2,2024-08-26 15:28:07,2024-08-26 16:46:57
19766,24476,3897.0,14.817847,-1284.0,79.112279,1,2024-09-06 19:06:55,2024-09-06 20:25:24
19767,18387,4578.0,18.834880,-1544.0,111.540439,1,2024-09-08 18:44:55,2024-09-08 20:42:48


In [102]:
final_time_df

,trip_id,trip_real_duration,trip_avg_speed,delay,avg_variance,direction_id,start_datetime,end_datetime
0,9633,4867.0,15.618442,-1038.0,76.544392,2,2024-08-13 18:54:32,2024-08-13 20:31:47
1,10336,3086.0,16.991997,-798.0,65.321028,1,2024-08-14 20:40:09,2024-08-14 21:43:17
2,11490,6773.0,15.321919,-1171.0,85.261477,1,2024-08-16 15:17:37,2024-08-16 18:02:56
3,11782,3322.0,17.375570,-1190.0,74.624023,1,2024-08-17 08:09:10,2024-08-17 09:19:13
4,5212,4036.0,18.395965,-1173.0,98.269678,2,2024-08-06 15:13:51,2024-08-06 16:40:55
...,...,...,...,...,...,...,...,...
19764,26667,3922.0,15.927480,-1241.0,101.851187,2,2024-09-13 13:24:11,2024-09-13 14:47:32
19765,17594,3756.0,16.981150,-1296.0,85.469122,2,2024-08-26 15:28:07,2024-08-26 16:46:57
19766,24476,3897.0,14.817847,-1284.0,79.112279,1,2024-09-06 19:06:55,2024-09-06 20:25:24
19767,18387,4578.0,18.834880,-1544.0,111.540439,1,2024-09-08 18:44:55,2024-09-08 20:42:48


In [106]:
final_time_df["month"] = final_time_df["start_datetime"].dt.month
final_time_df["is_weekend"] = final_time_df["start_datetime"].dt.dayofweek >= 5
final_time_df["hour"] = final_time_df["start_datetime"].dt.hour

In [110]:
final_time_df = final_time_df.drop(columns=["day_name"])

In [111]:
final_time_df

,trip_id,trip_real_duration,trip_avg_speed,delay,avg_variance,direction_id,start_datetime,end_datetime,is_weekend,hour
0,9633,4867.0,15.618442,-1038.0,76.544392,2,2024-08-13 18:54:32,2024-08-13 20:31:47,False,18
1,10336,3086.0,16.991997,-798.0,65.321028,1,2024-08-14 20:40:09,2024-08-14 21:43:17,False,20
2,11490,6773.0,15.321919,-1171.0,85.261477,1,2024-08-16 15:17:37,2024-08-16 18:02:56,False,15
3,11782,3322.0,17.375570,-1190.0,74.624023,1,2024-08-17 08:09:10,2024-08-17 09:19:13,True,8
4,5212,4036.0,18.395965,-1173.0,98.269678,2,2024-08-06 15:13:51,2024-08-06 16:40:55,False,15
...,...,...,...,...,...,...,...,...,...,...
19764,26667,3922.0,15.927480,-1241.0,101.851187,2,2024-09-13 13:24:11,2024-09-13 14:47:32,False,13
19765,17594,3756.0,16.981150,-1296.0,85.469122,2,2024-08-26 15:28:07,2024-08-26 16:46:57,False,15
19766,24476,3897.0,14.817847,-1284.0,79.112279,1,2024-09-06 19:06:55,2024-09-06 20:25:24,False,19
19767,18387,4578.0,18.834880,-1544.0,111.540439,1,2024-09-08 18:44:55,2024-09-08 20:42:48,True,18


In [113]:
# useful for regression, because model can understand time is cyclical 

final_time_df["hour_sin"] = np.sin(2 * np.pi * final_time_df["hour"] / 24)
final_time_df["hour_cos"] = np.cos(2 * np.pi * final_time_df["hour"] / 24)

In [115]:
final_time_df = final_time_df.drop(columns=["hour"])

In [117]:
final_time_df["day_of_week"] = final_time_df["start_datetime"].dt.dayofweek 


In [120]:
# same with weekday
final_time_df["dow_sin"] = np.sin(2 * np.pi * final_time_df["day_of_week"] / 7)
final_time_df["dow_cos"] = np.cos(2 * np.pi * final_time_df["day_of_week"] / 7)

In [130]:
final_time_df =pd.read_parquet("final_time_df.parquet")

In [131]:
final_time_df = final_time_df.merge(
    trip_tweak[["trip_id", "scheduled_duration"]],
    on="trip_id",
    how="left"
)

In [133]:

con.register("final_time_df", final_time_df)

con.execute("""
COPY final_time_df TO 'final_time_df.parquet' (FORMAT PARQUET);
""")

In [85]:
final_time_df =pd.read_parquet("final_time_df.parquet")

In [86]:
final_time_df

,trip_id,trip_real_duration,trip_avg_speed,delay,avg_variance,direction_id,start_datetime,end_datetime,is_weekend,hour_sin,hour_cos,day_of_week,dow_sin,dow_cos,scheduled_duration
0,9633,4867.0,15.618442,-1038.0,76.544392,2,2024-08-13 18:54:32,2024-08-13 20:31:47,False,-1.000000,-1.836970e-16,1,0.781831,0.623490,5905.0
1,10336,3086.0,16.991997,-798.0,65.321028,1,2024-08-14 20:40:09,2024-08-14 21:43:17,False,-0.866025,5.000000e-01,2,0.974928,-0.222521,3884.0
2,11490,6773.0,15.321919,-1171.0,85.261477,1,2024-08-16 15:17:37,2024-08-16 18:02:56,False,-0.707107,-7.071068e-01,4,-0.433884,-0.900969,7944.0
3,11782,3322.0,17.375570,-1190.0,74.624023,1,2024-08-17 08:09:10,2024-08-17 09:19:13,True,0.866025,-5.000000e-01,5,-0.974928,-0.222521,4512.0
4,5212,4036.0,18.395965,-1173.0,98.269678,2,2024-08-06 15:13:51,2024-08-06 16:40:55,False,-0.707107,-7.071068e-01,1,0.781831,0.623490,5209.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19764,26667,3922.0,15.927480,-1241.0,101.851187,2,2024-09-13 13:24:11,2024-09-13 14:47:32,False,-0.258819,-9.659258e-01,4,-0.433884,-0.900969,5163.0
19765,17594,3756.0,16.981150,-1296.0,85.469122,2,2024-08-26 15:28:07,2024-08-26 16:46:57,False,-0.707107,-7.071068e-01,0,0.000000,1.000000,5052.0
19766,24476,3897.0,14.817847,-1284.0,79.112279,1,2024-09-06 19:06:55,2024-09-06 20:25:24,False,-0.965926,2.588190e-01,4,-0.433884,-0.900969,5181.0
19767,18387,4578.0,18.834880,-1544.0,111.540439,1,2024-09-08 18:44:55,2024-09-08 20:42:48,True,-1.000000,-1.836970e-16,6,-0.781831,0.623490,6122.0


In [87]:
final_time_df = final_time_df.drop(columns=["trip_avg_speed", "delay", "day_of_week"])

In [88]:
final_time_df = final_time_df.merge(
    trips[["vehicle_id", "trip_id"]],
    how="left",
    on="trip_id"
)

In [109]:
final_time_df = final_time_df.sort_values(["vehicle_id", "start_datetime", "" ])


In [110]:
final_time_df

,trip_id,trip_real_duration,avg_variance,direction_id,start_datetime,end_datetime,is_weekend,hour_sin,hour_cos,dow_sin,dow_cos,scheduled_duration,vehicle_id,trd_shift_1,prev_end_shift_1,gap_minutes,time_diff
18897,100,3693.0,106.185268,2,2024-07-29 08:11:07,2024-07-29 09:33:10,False,8.660254e-01,-5.000000e-01,0.000000,1.000000,4908.0,010670bd-20f2-485a-b2d5-f4a5fae53208,5263.0,2024-09-20 12:16:24,-76565.283333,-1570.0
1604,156,5245.0,90.837144,1,2024-07-29 09:09:47,2024-07-29 11:20:16,False,7.071068e-01,-7.071068e-01,0.000000,1.000000,6601.0,010670bd-20f2-485a-b2d5-f4a5fae53208,2982.0,2024-07-29 07:54:04,75.716667,NaN
14148,238,2873.0,92.365032,2,2024-07-29 11:30:36,2024-07-29 12:34:12,False,2.588190e-01,-9.659258e-01,0.000000,1.000000,3911.0,010670bd-20f2-485a-b2d5-f4a5fae53208,3693.0,2024-07-29 09:33:10,117.433333,-820.0
4120,284,2881.0,104.250785,1,2024-07-29 12:46:15,2024-07-29 13:51:28,False,1.224647e-16,-1.000000e+00,0.000000,1.000000,4747.0,010670bd-20f2-485a-b2d5-f4a5fae53208,5245.0,2024-07-29 11:20:16,85.983333,-2364.0
15304,340,3298.0,96.952717,2,2024-07-29 14:05:12,2024-07-29 15:19:22,False,-5.000000e-01,-8.660254e-01,0.000000,1.000000,5085.0,010670bd-20f2-485a-b2d5-f4a5fae53208,2873.0,2024-07-29 12:34:12,91.000000,425.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12127,20577,3656.0,126.981175,1,2024-09-13 13:13:29,2024-09-13 14:32:50,False,-2.588190e-01,-9.659258e-01,-0.433884,-0.900969,7359.0,fc828ac8-2318-49d9-bb73-a9b1de024469,5356.0,2024-09-12 17:04:51,1208.633333,-1700.0
10599,20791,3380.0,142.811997,1,2024-09-13 20:04:44,2024-09-13 21:14:52,False,-8.660254e-01,5.000000e-01,-0.433884,-0.900969,4877.0,fc828ac8-2318-49d9-bb73-a9b1de024469,3656.0,2024-09-13 14:32:50,331.900000,-276.0
5589,20887,2739.0,142.182097,1,2024-09-14 06:00:28,2024-09-14 06:58:51,True,1.000000e+00,6.123234e-17,-0.974928,-0.222521,3825.0,fc828ac8-2318-49d9-bb73-a9b1de024469,3380.0,2024-09-13 21:14:52,525.600000,-641.0
4515,21103,3169.0,144.295080,1,2024-09-14 14:17:14,2024-09-14 15:39:48,True,-5.000000e-01,-8.660254e-01,-0.974928,-0.222521,4951.0,fc828ac8-2318-49d9-bb73-a9b1de024469,2739.0,2024-09-14 06:58:51,438.383333,430.0


In [92]:
# https://www.youtube.com/watch?v=F8YeVV3DNKg lag justification
final_time_df["trd_shift_1"] = final_time_df.groupby("vehicle_id")["trip_real_duration"].shift(1)
final_time_df["prev_end_shift_1"] = final_time_df.groupby("vehicle_id")["end_datetime"].shift(1)

final_time_df["gap_minutes"] = (
    final_time_df["start_datetime"] - final_time_df["prev_end_shift_1"]
).dt.total_seconds() / 60

In [93]:
final_time_df["gap_minutes"].quantile([0.25, 0.5, 0.75])

0.25     86.266667
0.50    115.800000
0.75    530.466667
Name: gap_minutes, dtype: float64

In [95]:
final_time_df = final_time_df.dropna()

In [96]:
final_time_df

,trip_id,trip_real_duration,avg_variance,direction_id,start_datetime,end_datetime,is_weekend,hour_sin,hour_cos,dow_sin,dow_cos,scheduled_duration,vehicle_id,trd_shift_1,prev_end_shift_1,gap_minutes
1604,156,5245.0,90.837144,1,2024-07-29 09:09:47,2024-07-29 11:20:16,False,7.071068e-01,-7.071068e-01,0.000000,1.000000,6601.0,010670bd-20f2-485a-b2d5-f4a5fae53208,2982.0,2024-07-29 07:54:04,75.716667
4120,284,2881.0,104.250785,1,2024-07-29 12:46:15,2024-07-29 13:51:28,False,1.224647e-16,-1.000000e+00,0.000000,1.000000,4747.0,010670bd-20f2-485a-b2d5-f4a5fae53208,5245.0,2024-07-29 11:20:16,85.983333
1605,402,5488.0,107.909038,1,2024-07-29 15:22:38,2024-07-29 17:11:08,False,-7.071068e-01,-7.071068e-01,0.000000,1.000000,6876.0,010670bd-20f2-485a-b2d5-f4a5fae53208,2881.0,2024-07-29 13:51:28,91.166667
12905,526,3809.0,99.464963,1,2024-07-29 18:44:46,2024-07-29 20:10:19,False,-1.000000e+00,-1.836970e-16,0.000000,1.000000,5556.0,010670bd-20f2-485a-b2d5-f4a5fae53208,5488.0,2024-07-29 17:11:08,93.633333
16481,672,2564.0,108.215301,1,2024-07-30 06:00:23,2024-07-30 06:57:06,False,1.000000e+00,6.123234e-17,0.781831,0.623490,3709.0,010670bd-20f2-485a-b2d5-f4a5fae53208,3809.0,2024-07-29 20:10:19,590.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18307,11553,3890.0,132.114941,2,2024-08-23 15:33:33,2024-08-23 16:57:42,False,-7.071068e-01,-7.071068e-01,-0.433884,-0.900969,5106.0,fc828ac8-2318-49d9-bb73-a9b1de024469,3414.0,2024-08-22 09:37:29,1796.066667
18308,12633,3104.0,136.557473,2,2024-08-26 10:10:53,2024-08-26 11:22:16,False,5.000000e-01,-8.660254e-01,0.000000,1.000000,4638.0,fc828ac8-2318-49d9-bb73-a9b1de024469,3890.0,2024-08-23 16:57:42,3913.183333
17895,19761,4639.0,158.819395,2,2024-09-11 18:05:50,2024-09-11 19:45:31,False,-1.000000e+00,-1.836970e-16,0.974928,-0.222521,6454.0,fc828ac8-2318-49d9-bb73-a9b1de024469,3104.0,2024-08-26 11:22:16,23443.566667
8393,20539,3173.0,149.774006,2,2024-09-13 11:24:44,2024-09-13 12:36:38,False,2.588190e-01,-9.659258e-01,-0.433884,-0.900969,4732.0,fc828ac8-2318-49d9-bb73-a9b1de024469,4639.0,2024-09-11 19:45:31,2379.216667


In [106]:
final_time_df["start_datetime"].dtype

dtype('<M8[us]')

In [104]:
assert (
    final_time_df.groupby("vehicle_id")["start_datetime"]
      .apply(lambda x: x.is_monotonic_increasing)
      .all()
)

AssertionError: 

In [7]:
X = final_time_df.drop(columns=["trip_real_duration"])
y = final_time_df["trip_real_duration"]